https://golang.org/doc/codewalk/markov/
https://en.wikipedia.org/wiki/Markov_chain
https://en.wikipedia.org/wiki/N-gram#n-gram_models

In [ ]:
from nltk import ngrams
sentence = 'this is a foo bar sentences and i want to ngramize it'
n = 3
sixgrams = ngrams(sentence.split(), n)
for grams in sixgrams:
  print grams

In [ ]:
sixgrams[0]

In [ ]:
import markovify

# Get raw text as string.
with open("lyrics.txt") as f:
    text = f.read()

# Build the model.
text_model = markovify.Text(text)

# Print five randomly-generated sentences
for i in range(5):
    print(text_model.make_sentence())

# Print three randomly-generated sentences of no more than 140 characters
for i in range(10):
    print(text_model.make_short_sentence(1000))

https://agiliq.com/blog/2009/06/generating-pseudo-random-text-with-markov-chains-u/

In [ ]:
file_ = open('lyrics.txt')

import markovgen

markov = markovgen.Markov(file_)
markov.generate_markov_text(max_size=10, seed="alma")